In [22]:
import os
import torch
import faiss
import logging
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from Libraries import A0_MyUtils as A0, A1_TextProcess as A1, A2_PdfProcess as A2
from Libraries import B1_ExtractData as B1, B2_MergeData as B2, B3_GetStructures as B3
from Libraries import B4_ChunkMaster as B4, B5_ChunkFlex as B5, B6_ChunkFixed as B6
from Libraries import C1_CreateSchema as C1, C2_Embedding as C2, C3_CheckStruct as C3, C4_FaissConvert as C4
from Libraries import D1_Search as D1, D2_Rerank as D2, D3_Respond as D3
from Config import Widgets, Configs, ModelLoader as ML

In [23]:
widgets_list = Widgets.create_name_form()

Dropdown(description='Model: ', index=1, layout=Layout(width='90%'), options=('Auto Model', 'Sentence Transfor…

Dropdown(description='Embedder: ', index=2, layout=Layout(width='90%'), options=('vinai/phobert-base', 'keepit…

Dropdown(description='Searcher: ', index=1, layout=Layout(width='90%'), options=('faiss.IndexHNSWFlat', 'faiss…

Dropdown(description='Reranker: ', layout=Layout(width='90%'), options=('BAAI/bge-reranker-base',), value='BAA…

Dropdown(description='Response: ', layout=Layout(width='90%'), options=('gemini-2.0-flash-exp', 'vinai/PhoGPT-…

Dropdown(description='API Key:', index=4, layout=Layout(width='90%'), options=('AIzaSyDaHS-8h6GJkyVPhoX4svvYeB…

In [24]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [ ]:
config = Configs.WidgetValues(widgets_list)

data_foler = config["data_folder"]
dcmt_path = config["dcmt_path"]
base_folder = config["base_folder"]
base_path = config["base_path"]
extracted_path = config["extracted_path"]
merged_path = config["merged_path"]
struct_path = config["struct_path"]
chunks_struct = config["chunks_struct"]
chunks_segment = config["chunks_segment"]
schema_ex_path = config["schema_ex_path"]
embedding_path = config["embedding_path"]
torch_path = config["torch_path"]
faiss_path = config["faiss_path"]
mapping_path = config["mapping_path"]
map_data_path = config["map_data_path"]
meta_path = config["meta_path"]

FILE_TYPE = config["FILE_TYPE"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SWITCH = config["SWITCH"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
API_KEY = config["API_KEY"]

WORD_LIMIT = config["WORD_LIMIT"]

SEARCH_ENGINE = faiss.IndexFlatIP

Model   : Sentence Transformer
Type    : Data
Embedder: VoVanPhuc/sup-SimCSE-VietNamese-phobert-base
Searcher: faiss.IndexFlatIP
Reranker: BAAI/bge-reranker-base
Responer: gemini-2.0-flash-exp
Data Key: contents
Embe Key: embeddings
File    : HNMU_Regulations
Dcment  : ../Docs/HNMU_Regulations.pdf
Extract : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Extracted.json
Merge   : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Merged.json
Struct  : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Struct.json
Chunked : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Chunks.json
Segment : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Texts_Segment.json
Torch   : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeddings.pt
Faiss   : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeddings.faiss
Mapping : ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeddings_mapping.json
Map Data: ../Data/HNMU_Regulations/Data_HNMU_Regulations_Embeddings_map_data.json
Meta    : ../D

## Prepare

### Fixed

In [26]:
# loaded model
cached_path = "../Models"

# assets
assets = "../Assets/"
exceptions_path = f"{assets}ex.exceptions.json"
markers_path = f"{assets}ex.markers.json"
status_path = f"{assets}ex.status.json"

### Device

In [27]:
ML.CudaCheck()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if SWITCH == "Auto Model":
    if os.path.exists(cached_path):
        tokenizer, model = ML.load_auto_model(cached_path, device)
        print(f"ℹ️ Auto Model: {cached_path}")
        if model is None:
            tokenizer, model = ML.load_auto_model(EMBEDD_MODEL, device)
    else:
        print(f"ℹ️ Auto Model: {EMBEDD_MODEL}")
        tokenizer, model = ML.load_auto_model(EMBEDD_MODEL, device)

elif SWITCH == "Sentence Transformer":
    if os.path.exists(cached_path):
        model = ML.load_sentence_model(cached_path, device)
        print(f"ℹ️ Sentece Transformer: {cached_path}")

        if model is None:
            model = ML.load_sentence_model(EMBEDD_MODEL, device)
    else:
        print(f"ℹ️ Sentece Transformer: {EMBEDD_MODEL}")
        model = ML.load_sentence_model(EMBEDD_MODEL, device)

print(f"✅ Using: {device}")

CUDA supported: True
Number of GPUs: 1
Current GPU name: NVIDIA GeForce RTX 2050
CUDA device capability: (8, 6)
CUDA version (PyTorch): 12.1
cuDNN version: 90100
ℹ️ Sentece Transformer: VoVanPhuc/sup-SimCSE-VietNamese-phobert-base
✅ Using: cuda


### Import Classes

In [28]:
dataExtractor = B1.B1Extractor(
    exceptions_path, 
    markers_path, 
    status_path, 
    proper_name_min_count=10
)

In [29]:
structAnalyzer = B3.StructureAnalyzer(
    merged_path = merged_path,
    verbose=True
)

In [30]:
chunkBuilder = B4.ChunkBuilder(
    struct_path=struct_path,
    merged_path=merged_path,
)

In [31]:
schemaEx = C1.JSONSchemaExtractor(
    list_policy="first", 
    verbose=True
)

In [32]:
Embedding = C2.JSONEmbedding(
    model=model,
    device="cuda:0",
    batch_size=32,
    show_progress=False,
    flatten_mode="split"
)

In [33]:
FaissConverter = C4.Torch2FaissConverter(
    schema_ex_path=schema_ex_path,
    torch_path=torch_path,
    faiss_path=faiss_path,
    mapping_path=mapping_path,
    map_data_path=map_data_path,
    keep_last=2,
    nlist=100,
    mode=EMBE_KEY,
    use_pickle=False
)

2025-09-30 16:13:58,380 - INFO - Chỉ giữ 2 field cuối trong schema: {'Article', 'Content'}


### Extract Data

In [34]:
def extractRun():
    extracted_data = dataExtractor.extract(dcmt_path)
    A0.write_json(extracted_data, extracted_path, indent=1)

    merged_data = B2.mergeLinesToParagraphs(extracted_data)
    A0.write_json(merged_data, merged_path, indent=1)

### Get Struct

In [35]:
def structRun():
    markers = structAnalyzer.extract_markers()

    structures = structAnalyzer.build_structures(markers)
    print(A0.jsonl_convert(structures))

    dedup = structAnalyzer.deduplicate(structures)
    print(A0.jsonl_convert(dedup))

    top = structAnalyzer.select_top(dedup)
    topext = structAnalyzer.extend_top(top, dedup)
    print(A0.json_convert(topext, pretty=True))

    A0.write_json(topext, struct_path, indent=2)

### Chunks

In [36]:
def chunkRun():
    chunks = chunkBuilder.build()
    A0.write_json(chunks, chunks_base, indent=2)

    filtered = [item for item in chunks if item.get("Level 1", "").strip()]
    for i, item in enumerate(filtered, start=1):
        item["Index"] = i

    A0.write_json(filtered, chunks_segment, indent=2)

### Schema Extract

In [37]:
def schemaRun():
    if os.path.exists(chunks_segment):
        schemaEx.schemaRun(chunks_segment, schema_path=schema_ex_path)
        chunksSchema = A0.read_json(schema_ex_path)
        print(chunksSchema)
    else:
        print(f"{chunks_segment} does not exist")

### Embedding

In [38]:
def embeddingRun():
    if os.path.exists(chunks_segment):
        Embedding.embeddingRun(
            json_path = chunks_segment,
            schema_path = schema_ex_path,
            torch_path = torch_path,
            data_key = DATA_KEY,
            embe_key = EMBE_KEY,
            skip_if_exists = False,
        )
    
    else:
        print(f"{chunks_segment} does not exist")

### Run

In [45]:
# if FILE_TYPE == "Data":
#     extractRun()
#     structRun()
#     chunkRun()
    
# schemaRun()
# embeddingRun()

C3.print_json(DATA_KEY, EMBE_KEY, torch_path, index = 12)
# FaissConverter.convert()

[contents] Tổng số mục: 468 — In mục index=12
{
  "Index": 13,
  "Level 1": "Chương I NHỮNG QUY ĐỊNH CHUNG",
  "Level 2": "Điều 4. Hình thức đào tạo",
  "Article": "1. Đào tạo chính quy",
  "Content": [
    "a) Các hoạt động giảng dạy được thực hiện tại trường, riêng những hoạt động thực hành, thực tập, trải nghiệm thực tế và giảng dạy trực tuyến có thể thực hiện ngoài trường;",
    "b) Thời gian tổ chức hoạt động giảng dạy trong khoảng từ 06 giờ đến 20 giờ các ngày trong tuần từ thứ 2 đến thứ 7;",
    "c) Thời gian tổ chức những hoạt đào tạo đặc thù khác được thực hiện theo quy định của Hiệu trưởng."
  ],
  "Level 1 Embedding": {
    "__vector_len__": 768,
    "head": [
      -0.2727910876274109,
      -0.008815515786409378,
      -0.12173040211200714,
      -0.7325150370597839,
      -0.3437274396419525
    ]
  },
  "Level 2 Embedding": {
    "__vector_len__": 768,
    "head": [
      -0.2813873291015625,
      -0.000455707311630249,
      0.040678106248378754,
      -0.8520454168319

## RAG

### Funcs

In [40]:
def searchRun(user_question):
    preliminary_results = D1.search_faiss_index(
        query = user_question,
        embedd_model = EMBEDD_MODEL,
        faiss_path = faiss_path,
        mapping_path = mapping_path,
        mapping_data = map_data_path,
        device = device,
        k = 2,
        min_score = 5,
        batches = False,
    )
    return preliminary_results

def rerankRun(user_question, preliminary_results):
    reranked_results = D2.rerank_results(
        query = user_question,
        results = preliminary_results,
        reranker_model = RERANK_MODEL,
        device = device,
        k = 5,
        batches = False,
    )
    context = '\n\n'.join(item['text'] for item in reranked_results)
    return reranked_results, context

def respondRun(user_question, prompt, context="", doc=False):
    response = D3.respond_naturally(
        user_question = user_question,
        context = context,
        prompt = prompt,
        respond_model = RESPON_MODEL,
        score_threshold = 0.85,
        max_results = 3,
        doc = doc,
        gemini_api_key = API_KEY,
    )
    return response

### Main

In [41]:
prompt_type = "Docs"

queries = [
    "Quy chế này quy định những gì và áp dụng cho đối tượng nào",
    "Sinh viên có thể được thi lại bao nhiêu lần?",
]

def chatRun():

    prompt_path = f"Prompts/{prompt_type}_Prompt.txt"
    with open(prompt_path, "r", encoding="utf-8") as f: prompt = f.read()

    print("<< Enter 'exit', 'quit', 'escape', 'bye' or Press ESC to exit >>")
    print("Chatbot: Hello there! I'm here to help you!\n\n")

    query = -1
    while True:
        try:
            query += 1
            if query >= len(queries):
                user_input = "exit"
            else:
                user_input = queries[query]

            # user_input = input("You: ")

            user_question = A0.preprocess_text(user_input)
            if user_input.strip().lower() in ["exit", "quit", "escape", "bye", ""]:
                print("Chatbot: Goodbye!")
                break

            print(f"Query: {user_question}")

            # Bước 1: Search
            preliminary_results = searchRun(user_question)
            
            # Bước 2: Rerank
            if preliminary_results:
                print(preliminary_results)
                reranked_results, context = rerankRun(user_question, preliminary_results)
            else:
                print("Không tìm thấy thông tin!")

            # Bước 3: Generate Response
            if (reranked_results):
                print(reranked_results)
                print(f"\n Context:\n {context}")
                response = respondRun(user_question, prompt, context=context, doc=True)
            else:
                print("Rerank thất bại!")

            # Bước 4: Print Response
            if response:
                print(f"\nYou: {user_question}")
                print(f"Chatbot: {response}\n\n")
            else:
                print("LLM không phản hồi!")

            print("=" * 200)
            print("\n\n")

        except KeyboardInterrupt:
            print("\nChatbot: Goodbye!")
            break

In [42]:
# chatRun()